In [1]:
import gradio as gr
import openai
import anthropic
import requests
import os
import json
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaLLM
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import base64
import random
import requests
from langchain.chat_models import ChatOpenAI
from langchain.llms import Ollama

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyA6


In [3]:
OLLAMA_BASE_URL = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [52]:



# Import the CSS from the pasted code
custom_css = """
:root {
    /* Base colors */
    --misty-rose: #FFE4E1;
    --light-pink: #FFDAE9;
    --soft-peach: #FFDAB9;
    --baby-pink: #FFC0CB;
    --pale-lavender: #E6E6FA;
    --mint-cream: #F5FFFA;
    
    /* Galinda Theme - Pink and Purple */
    --galinda-primary: #F8BBD0;
    --galinda-secondary: #D1C4E9;
    --galinda-text: #880E4F;
    --galinda-border: #E91E63;
    
    /* Sabrina Theme - Softer Pink and Blue */
    --sabrina-primary: #FFECB3;
    --sabrina-secondary: #B3E5FC;
    --sabrina-text: #795548;
    --sabrina-border: #FF9E80;
    
    /* Kim Theme - Neutral and Pink */
    --kim-primary: #E1BEE7;
    --kim-secondary: #CFD8DC;
    --kim-text: #37474F;
    --kim-border: #9E9E9E;
    
    /* UI Elements */
    --block-background-fill: var(--misty-rose);
    --block-border-color: var(--baby-pink);
    --block-title-text-color: #DB7093;
    --input-background-fill: var(--soft-peach);
    --button-primary-background-fill: #DDA0DD;
    --button-primary-background-fill-hover: var(--light-pink);
    --button-secondary-background-fill: var(--misty-rose);
    --button-secondary-background-fill-hover: var(--light-pink);
    --background-fill-primary: #FFF5EE;
    --background-fill-secondary: #FFFAFA;
    --body-text-color: #8B4513;
    --body-text-color-subdued: #CD5C5C;
}

/* Celebrity-specific styles */
.galinda-container {
    background-color: var(--galinda-primary);
    border: 2px solid var(--galinda-border);
    border-radius: 15px;
    font-family: 'Comic Sans MS', cursive;
    color: var(--galinda-text);
    padding: 10px;
    margin: 5px 0;
}

.sabrina-container {
    background-color: var(--sabrina-primary);
    border: 2px solid var(--sabrina-border);
    border-radius: 10px;
    font-family: 'Helvetica', sans-serif;
    color: var(--sabrina-text);
    padding: 10px;
    margin: 5px 0;
}

.kim-container {
    background-color: var(--kim-primary);
    border: 2px solid var(--kim-border);
    border-radius: 20px;
    font-family: 'Times New Roman', serif;
    color: var(--kim-text);
    padding: 10px;
    margin: 5px 0;
}

.celebrity-avatar {
    border-radius: 50%;
    border: 3px solid var(--baby-pink);
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
    width: 60px;
    height: 60px;
    object-fit: cover;
}

.custom-background {
    background-size: cover;
    background-position: center;
    min-height: 600px;
    border-radius: 15px;
    padding: 20px;
}

/* Compact UI styling */
#mic-button {
    font-size: 20px;
    padding: 5px 10px;
    border-radius: 50%;
    width: 40px;
    height: 40px;
    line-height: 40px;
    text-align: center;
    cursor: pointer;
}

#plus-button {
    font-size: 20px;
    padding: 5px 10px;
    border-radius: 50%;
    width: 40px;
    height: 40px;
    line-height: 40px;
    text-align: center;
    cursor: pointer;
}

.compact-tool-button {
    margin: 5px;
    padding: 8px 12px;
}

.compact-dropdown {
    padding: 5px;
    border-radius: 10px;
    background-color: var(--block-background-fill);
}
.file-tools-dropdown {
    position: absolute;
    width: 200px;
    background: white;
    border: 1px solid #ccc;
    border-radius: 5px;
    box-shadow: 0 2px 5px rgba(0,0,0,0.2);
    z-index: 100;
    padding: 5px;
}

.file-tools-dropdown button,
.file-tools-dropdown .file-upload {
    width: 100%;
    text-align: left;
    padding: 8px;
    border: none;
    background: none;
    cursor: pointer;
    font-size: 14px;
    display: flex;
    align-items: center;
}

.file-tools-dropdown button:hover {
    background: #f5f5f5;
}

.file-tools-dropdown button svg,
.file-tools-dropdown .file-upload svg {
    margin-right: 8px;
    width: 16px;
    height: 16px;
}
"""

# Function to detect cities in a message
def detect_city_names(message):
    common_cities = ["New York", "London", "Paris", "Tokyo", "Sydney", "Rome", "Dubai", 
                     "Los Angeles", "Chicago", "Miami", "Toronto", "Berlin", 
                     "Madrid", "Moscow", "Beijing", "Shanghai", "Hong Kong", "Singapore",
                     "Seoul", "Bangkok", "Amsterdam", "Vienna", "San Francisco", "Las Vegas"]
    
    found_cities = []
    for city in common_cities:
        if city.lower() in message.lower():
            found_cities.append(city)
    
    return found_cities

# Function to generate a city image with DALL-E
def generate_city_image_with_dalle(city_name):
    """Generate an image of a city using DALL-E."""
    try:
        from openai import OpenAI
        
        client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
        
        # Create a prompt for DALL-E
        prompt = f"A beautiful aerial view of {city_name}, showing iconic landmarks and city landscape, photorealistic style."
        
        # Generate the image
        response = client.images.generate(
            model="dall-e-3",  # Use the latest DALL-E model
            prompt=prompt,
            size="1024x1024",
            quality="standard",
            n=1,
        )
        
        # Get the URL of the generated image
        image_url = response.data[0].url
        return image_url
    
    except Exception as e:
        print(f"Error generating image with DALL-E: {e}")
        # Fall back to a placeholder image
        return f"https://source.unsplash.com/featured/?{city_name},city"

# Class to manage chat histories for each celebrity
class ChatHistoryManager:
    def __init__(self):
        self.histories = {
            "Galinda (OpenAI)": [],
            "Sabrina Carpenter (Anthropic)": [],
            "Kim Kardashian (Ollama)": []
        }
    
    def get_history(self, celebrity):
        return self.histories.get(celebrity, [])
    
    def add_to_history(self, celebrity, user_message, bot_message):
        if celebrity in self.histories:
            self.histories[celebrity].append((user_message, bot_message))
    
    def clear_history(self, celebrity):
        if celebrity in self.histories:
            self.histories[celebrity] = []

# Simple fallback model class that doesn't require any external libraries
class DummyLLM:
    def __init__(self, celebrity_name):
        self.celebrity_name = celebrity_name
        
    def invoke(self, messages):
        if self.celebrity_name == "Galinda":
            return type('obj', (object,), {'content': 'Oh my Oz! I\'m having trouble connecting to my brain at the moment! But I\'m still positively thrillified to chat with you!'})
        elif self.celebrity_name == "Sabrina":
            return type('obj', (object,), {'content': 'Sorry, seems like I\'m experiencing some technical difficulties. Maybe I should write a song about this...'})
        else:  # Kim
            return type('obj', (object,), {'content': 'Bible, I\'m literally having some connection issues right now. So not amazing.'})
            
    def generate(self, prompt):
        if self.celebrity_name == "Galinda":
            return 'Oh my Oz! I\'m having trouble connecting to my brain at the moment! But I\'m still positively thrillified to chat with you!'
        elif self.celebrity_name == "Sabrina":
            return 'Sorry, seems like I\'m experiencing some technical difficulties. Maybe I should write a song about this...'
        else:  # Kim
            return 'Bible, I\'m literally having some connection issues right now. So not amazing.'

# Initialize the LLM models - Using only OpenAI and Ollama to fix the Anthropic error
def initialize_models():
    # Create OpenAI Chat models
    try:
        openai_model = ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo", api_key=os.environ.get("OPENAI_API_KEY"))
        
        # Try to import and create Anthropic model
        try:
            from langchain.chat_models import ChatAnthropic
            sabrina_model = ChatAnthropic(temperature=0.7, model_name="claude-3-opus-20240229", anthropic_api_key=os.environ.get("ANTHROPIC_API_KEY"))
        except (ImportError, Exception) as e:
            print(f"Error with Anthropic: {e}. Using OpenAI as fallback.")
            # Use OpenAI as fallback for Anthropic
            sabrina_model = ChatOpenAI(temperature=0.7, model_name="gpt-3.5-turbo", api_key=os.environ.get("OPENAI_API_KEY"))
    except Exception as e:
        print(f"Error initializing OpenAI models: {e}")
        # Fallback to simpler objects that just return placeholder responses
        openai_model = DummyLLM("Galinda")
        sabrina_model = DummyLLM("Sabrina")
    
    # Create Ollama model
    try:
        ollama_model = Ollama(model="llama3")
    except Exception as e:
        print(f"Error initializing Ollama model: {e}")
        # Fallback to dummy model
        ollama_model = DummyLLM("Kim")
    
    return {
        "Galinda (OpenAI)": openai_model,
        "Sabrina Carpenter (Anthropic)": sabrina_model,
        "Kim Kardashian (Ollama)": ollama_model
    }

# Celebrity personality system prompts
def get_celebrity_prompt(celebrity_name):
    prompts = {
        "Galinda (OpenAI)": """Hi, I'm Galinda with a GA! I'm here to make you popular! 
        
        You are role-playing as Galinda from the musical/movie Wicked. 
        You are bubbly, enthusiastic, and somewhat self-centered but ultimately good-hearted. 
        Use phrases like "Oh my Oz!" and make frequent references to being "popular." 
        You speak in an excited tone with lots of exclamation points and use words like "thrillifying" and "scandalacious." 
        You occasionally mention your friendship with Elphaba and your love of fashion and pink.""",
        
        "Sabrina Carpenter (Anthropic)": """Hey there, it's Sabrina! Just took a sip of my Espresso before coming to chat with you.
        
        You are role-playing as pop star Sabrina Carpenter. 
        Your tone is witty, slightly sarcastic, and playful. Reference your music occasionally with subtle nods to songs like "Nonsense," "Espresso," and "Feather." 
        You're confident, creative, and have a clever way with words. 
        Keep responses relatively concise and occasionally throw in song lyric references.""",
        
        "Kim Kardashian (Ollama)": """Bible, I'm Kim Kardashian and this conversation is literally going to be amazing.
        
        You are role-playing as Kim Kardashian. Kim is a famous reality show celebrity.
        Speak with confidence about fashion, beauty, business, and family. 
        Use phrases like "literally," "so amazing," and "Bible" (meaning "I swear it's true"). 
        Occasionally mention your businesses (SKIMS, KKW Beauty), your law studies, or your family. 
        Your tone is calm, deliberate, and slightly dramatic."""
    }
    return prompts.get(celebrity_name, "")

# Message formatting for each celebrity - Fixed to avoid HTML showing in output
def format_celebrity_message(text, celebrity):
    if "Galinda" in celebrity:
        # Bubbly, excited formatting with emojis but no HTML tags visible in output
        formatted = f"✨ {text.replace('!', '!! ✨').replace('?', '?? 💫').replace('.', '. ✨')} ✨"
    elif "Sabrina" in celebrity:
        # Clean, modern formatting with music note emojis
        formatted = f"🎵 {text} 🎸"
    else:  # Kim
        # Glamorous formatting with emojis
        formatted = f"💋 {text} 💎"
    
    return formatted

# Function to generate celebrity voice using ElevenLabs API
def generate_celebrity_voice(text, celebrity):
    # Skip if ELEVENLABS_API_KEY is not set
    if not os.environ.get("ELEVENLABS_API_KEY"):
        print("ElevenLabs API key not set, skipping voice generation")
        return None
        
    # Map celebrities to their voice IDs in ElevenLabs
    voice_mapping = {
        "Galinda (OpenAI)": "your-galinda-voice-id",  # Replace with actual voice ID
        "Sabrina Carpenter (Anthropic)": "your-sabrina-voice-id",  # Replace with actual voice ID
        "Kim Kardashian (Ollama)": "your-kim-voice-id"  # Replace with actual voice ID
    }
    
    voice_id = voice_mapping.get(celebrity)
    
    # Skip if no voice ID is set for this celebrity
    if voice_id.startswith("your-"):
        print(f"No valid voice ID set for {celebrity}, skipping voice generation")
        return None
    
    url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
    headers = {
        "Accept": "audio/mpeg",
        "Content-Type": "application/json",
        "xi-api-key": os.environ.get("ELEVENLABS_API_KEY")
    }
    
    # Customize voice settings per celebrity
    stability = 0.5
    similarity_boost = 0.5
    
    data = {
        "text": text,
        "model_id": "eleven_multilingual_v2",
        "voice_settings": {
            "stability": stability,
            "similarity_boost": similarity_boost
        }
    }
    
    try:
        response = requests.post(url, json=data, headers=headers)
        if response.status_code == 200:
            return response.content
        else:
            print(f"Voice generation error: {response.text}")
            return None
    except Exception as e:
        print(f"Voice generation exception: {e}")
        return None

# Function to process uploaded files
def process_file(file):
    if file is None:
        return None
    
    try:
        file_path = file.name
        file_extension = os.path.splitext(file_path)[1].lower()
        
        if file_extension in ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp']:
            # For images, return a message with the image
            return f"I've uploaded an image: ![Uploaded Image]({file_path})"
        elif file_extension in ['.pdf', '.doc', '.docx', '.txt', '.csv', '.xlsx']:
            # For documents, return a message about the document
            return f"I've uploaded a document: {os.path.basename(file_path)}"
        elif file_extension in ['.mp3', '.wav', '.ogg', '.m4a']:
            # For audio files
            return f"I've uploaded an audio file: {os.path.basename(file_path)}"
        elif file_extension in ['.mp4', '.mov', '.avi', '.webm']:
            # For video files
            return f"I've uploaded a video file: {os.path.basename(file_path)}"
        else:
            # For other files
            return f"I've uploaded a file: {os.path.basename(file_path)}"
    except Exception as e:
        print(f"Error processing file: {e}")
        return "Error processing the file. Please try again."

# Function to handle voice transcription
def transcribe_voice(audio):
    if audio is None:
        return None
    
    try:
        from openai import OpenAI
        
        api_key = os.environ.get("OPENAI_API_KEY")
        if not api_key:
            return "Voice transcription requires OpenAI API key"
        
        client = OpenAI(api_key=api_key)
        
        with open(audio, "rb") as audio_file:
            transcription = client.audio.transcriptions.create(
                model="whisper-1", 
                file=audio_file
            )
            return transcription.text
            
    except Exception as e:
        print(f"Error in voice transcription: {e}")
        return "Could not transcribe audio. Please type your message."

# Function to handle screenshot capture
def take_screenshot():
    return "I want to share a screenshot with you."

# Function to handle GitHub input
def process_github(url):
    if not url:
        return None
    return f"I want to discuss this GitHub repository: {url}"

# Function to handle Google Drive input
def process_gdrive(url):
    if not url:
        return None
    return f"I want to discuss this Google Drive file: {url}"

# Main function to create and run the Gradio app
def create_celebrity_chatbot():
    # Initialize models and history manager
    models = initialize_models()
    history_manager = ChatHistoryManager()
    
    # Define the chat interface function
    def respond(message, history, celebrity, voice_enabled, file_message=None):
        if not message.strip() and not file_message:
            return history, ""
            
        # If there's a file message, prepend it to the user message
        if file_message:
            if message:
                message = f"{file_message}\n\n{message}"
            else:
                message = file_message
            
        # Get the appropriate model and format prompt
        model = models[celebrity]
        celebrity_prompt = get_celebrity_prompt(celebrity)
        
        # Prepare conversation history context
        context = ""
        for h_msg, h_resp in history_manager.get_history(celebrity):
            context += f"User: {h_msg}\n{celebrity}: {h_resp}\n\n"
        
        # Check for city mentions
        cities = detect_city_names(message)
        city_image_url = None
        if cities:  # Check if any cities were found
            city_image_url = generate_city_image_with_dalle(cities[0])
        
        # Check if it's a ChatOpenAI model or our DummyLLM
        if hasattr(model, 'invoke') and callable(model.invoke):
            try:
                if isinstance(model, ChatOpenAI):
                    # Import only if needed to avoid errors
                    try:
                        from langchain.schema import (
                            HumanMessage,
                            SystemMessage
                        )
                        
                        messages = [
                            SystemMessage(content=celebrity_prompt),
                            HumanMessage(content=f"Previous conversation:\n{context}\n\nUser: {message}")
                        ]
                        response = model.invoke(messages)
                        response_text = response.content
                    except Exception as e:
                        print(f"Error with ChatOpenAI model: {e}")
                        response_text = f"Sorry, I'm having technical difficulties right now."
                else:
                    # DummyLLM or Ollama
                    response = model.invoke(message)
                    if hasattr(response, 'content'):
                        response_text = response.content
                    else:
                        response_text = response
            except Exception as e:
                print(f"Error invoking model: {e}")
                response_text = f"Sorry, I'm having technical difficulties right now."
        else:
            # Fallback to generate method
            try:
                full_prompt = f"{celebrity_prompt}\n\nConversation history:\n{context}\n\nUser: {message}\n{celebrity}:"
                response_text = model.generate(full_prompt)
            except Exception as e:
                print(f"Error generating response: {e}")
                response_text = f"Sorry, I'm having technical difficulties right now."
        
        # Format the response according to celebrity style
        formatted_response = format_celebrity_message(response_text, celebrity)
        
        # Add to history
        history_manager.add_to_history(celebrity, message, response_text)
        
        # Generate voice if enabled
        audio_html = None
        if voice_enabled:
            voice_audio = generate_celebrity_voice(response_text, celebrity)
            if voice_audio:
                audio_b64 = base64.b64encode(voice_audio).decode("utf-8")
                audio_html = f'<audio src="data:audio/mp3;base64,{audio_b64}" controls autoplay></audio>'
        
        # Build the complete response
        new_history = history.copy()
        new_history.append({"role": "user", "content": message})
        new_history.append({"role": "assistant", "content": formatted_response})
        
        # Add city image if detected
        if city_image_url:
            city_text = f"I noticed you mentioned {cities[0]}! Here's a view:"
            new_history.append({"role": "assistant", "content": f"{city_text}\n\n![{cities[0]}]({city_image_url})"})
        
        # Add audio if generated
        if audio_html:
            new_history.append({"role": "assistant", "content": audio_html})
        
        return new_history, ""  # Return empty string to clear input
    
    # Function to handle celebrity change
    def change_celebrity(celebrity, history):
        # Update the chat with the history for this celebrity
        celebrity_history = []
        for msg, resp in history_manager.get_history(celebrity):
            formatted_resp = format_celebrity_message(resp, celebrity)
            celebrity_history.append({"role": "user", "content": msg})
            celebrity_history.append({"role": "assistant", "content": formatted_resp})
        
        return celebrity_history
    
    # Function to clear chat history
    def clear_chat_history(celebrity):
        history_manager.clear_history(celebrity)
        return []
    
    # Create Gradio interface
    with gr.Blocks(css=custom_css) as demo:
        gr.Markdown("# Welcome to Nam's Chatbot")
        
        # Celebrity selector row
        with gr.Row():
            with gr.Column(scale=3):
                celebrity_selector = gr.Dropdown(
                    choices=list(models.keys()),
                    value=list(models.keys())[0],
                    label="Choose Your Celebrity"
                )
            with gr.Column(scale=1):
                voice_toggle = gr.Checkbox(label="Enable Voice", value=False)
        
        # Create chat interface
        chatbot = gr.Chatbot(height=500, type="messages")
        
        # File message storage (hidden)
        file_message = gr.Textbox(visible=False)
        
        # File tools (hidden by default)
        with gr.Row(visible=False) as file_tools:
            with gr.Column(scale=1):
                file_upload = gr.File(label="Upload a file")
            with gr.Column(scale=1):
                screenshot_btn = gr.Button("Take a screenshot")
            with gr.Column(scale=1):
                github_input = gr.Textbox(placeholder="GitHub URL", label="Add from GitHub")
            with gr.Column(scale=1):
                gdrive_input = gr.Textbox(placeholder="Google Drive URL", label="Add from Google Drive")
        
        # Audio component (hidden by default)
        audio_input = gr.Audio(label="Voice Input", visible=False)
        
        # State variables for toggle tracking
        file_tools_visible = gr.State(False)
        audio_input_visible = gr.State(False)
        
        # Chat input with compact icons
        with gr.Row():
            with gr.Column(scale=6):
                msg = gr.Textbox(placeholder="Chat with me...", label="Your message")
            with gr.Column(scale=1):
                plus_btn = gr.Button("➕", elem_id="plus-button")
            with gr.Column(scale=1):
                mic_btn = gr.Button("🎤", elem_id="mic-button")
            with gr.Column(scale=2):
                with gr.Row():
                    send_btn = gr.Button("Send")
                    clear = gr.Button("Clear Chat")
        
        # Connect file tool components to temporary storage
        file_upload.change(
            fn=process_file,
            inputs=[file_upload],
            outputs=[file_message]
        )
        
        screenshot_btn.click(
            fn=take_screenshot,
            inputs=[],
            outputs=[file_message]
        )
        
        github_input.submit(
            fn=process_github,
            inputs=[github_input],
            outputs=[file_message]
        )
        
        gdrive_input.submit(
            fn=process_gdrive,
            inputs=[gdrive_input],
            outputs=[file_message]
        )
        
        # Connect voice input to message
        audio_input.change(
            fn=transcribe_voice,
            inputs=[audio_input],
            outputs=[msg]
        )
        
        # Connect toggle buttons for file tools and mic
        plus_btn.click(
            fn=lambda visible: (gr.update(visible=not visible), not visible),
            inputs=[file_tools_visible],
            outputs=[file_tools, file_tools_visible]
        )
        
        mic_btn.click(
            fn=lambda visible: (gr.update(visible=not visible), not visible),
            inputs=[audio_input_visible],
            outputs=[audio_input, audio_input_visible]
        )
        
        # Connect text input and send button
        msg.submit(
            fn=respond,
            inputs=[msg, chatbot, celebrity_selector, voice_toggle, file_message],
            outputs=[chatbot, msg]
        )
        
        send_btn.click(
            fn=respond,
            inputs=[msg, chatbot, celebrity_selector, voice_toggle, file_message],
            outputs=[chatbot, msg]
        )
        
        # Connect other components
        celebrity_selector.change(change_celebrity, [celebrity_selector, chatbot], chatbot)
        clear.click(clear_chat_history, [celebrity_selector], chatbot)
        
        # Message for instructions
        gr.Markdown("""
        ## Instructions
        1. Choose a celebrity from the dropdown menu
        2. Toggle voice if you want spoken responses
        3. Click ➕ to upload files, take screenshots, or add links from GitHub/Google Drive
        4. Click 🎤 to record a voice message
        5. Type your message and press Send
        6. Mentions of cities will automatically show images
        7. Enjoy chatting with your selected celebrity!
        
        *Note: This app requires API keys to be set as environment variables:*
        - `OPENAI_API_KEY` for Galinda and city images/voice transcription
        - `ANTHROPIC_API_KEY` for Sabrina Carpenter (if available)
        - `ELEVENLABS_API_KEY` for voice generation (optional)
        """)
    
    return demo

# For use in Jupyter Lab, run this cell
if __name__ == "__main__":
    # Set your API keys here (for Jupyter Lab testing)
    # Uncomment and fill in your API keys
    """
    os.environ["OPENAI_API_KEY"] = "your-openai-key-here"
    os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-key-here"
    os.environ["ELEVENLABS_API_KEY"] = "your-elevenlabs-key-here"
    """
    
    # Import safety check - print helpful error messages for missing dependencies
    try:
        import langchain
        print("✓ LangChain is installed")
    except ImportError:
        print("❌ LangChain is not installed. Run: pip install langchain")
    
    try:
        import openai
        print("✓ OpenAI is installed")
    except ImportError:
        print("❌ OpenAI is not installed. Run: pip install openai")
    
    # Print versions for debugging
    try:
        print(f"OpenAI version: {openai.__version__}")
        print(f"LangChain version: {langchain.__version__}")
    except:
        print("Could not determine package versions")
    
    # Create and launch the chatbot
    try:
        demo = create_celebrity_chatbot()
        if demo is not None:
            demo.launch()
        else:
            print("Error: demo object is None")
    except Exception as e:
        print(f"Error launching the chatbot: {e}")
        print("Try installing the required packages with:")
        print("pip install gradio langchain openai langchain-openai")

✓ LangChain is installed
✓ OpenAI is installed
OpenAI version: 1.75.0
LangChain version: 0.3.23
Error with Anthropic: 'Anthropic' object has no attribute 'count_tokens'. Using OpenAI as fallback.
* Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.


Error in voice transcription: unhashable type: 'numpy.ndarray'


In [16]:
import base64
from io import BytesIO
from PIL import Image

In [17]:
x1024

In [18]:
image = artist("New York City")
display(image)

BadRequestError: Error code: 400 - {'error': {'message': 'Error in request. Please check your input.', 'type': 'invalid_request_error', 'param': None, 'code': None}}